In [78]:
# !pip install matplotlib
# !pip install numpy
# !pip install pandas


In [79]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3

In [80]:
# se crea una base de datos en SQLite llamada veterinaria.db
conn = sqlite3.connect('../DataSet/Veterinaria.db')
# Creamos un cursor para ejecutar comandos SQL
cursor = conn.cursor()

In [81]:
def Funcion_ejecutar_DDL(query_ddl, conexion=conn):
    """
    Funcion encargada de ejecutar  sentencias  de tipo DDL(Definición de Datos - Define o modifica la estructura de la base de datos - CREATE, ALTER, DROP) 
    """
    cursor = conexion.cursor()
    cursor.execute(query_ddl)
    conexion.commit()
    return f"DDL ejecutado correctamente: {query_ddl}"

In [82]:
# Definimos una Función Auxiliar encargada de ejecutar consultas SQL
# Al llamar la consulta se recivira una tabla  en un DataFrame de pandas
def Funcion_ejecutar_DQL(query_dql, conexion=conn):
  """
  Funcion encargada de ejecutar  sentencias  de tipo DQL(Consulta de Datos - Recupera información - SELECT) 
  Encargada de devuer el resultado en un dataframe de pandas 
  """
  df = pd.read_sql_query(query_dql, conexion)
  return df


In [83]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS par_propietario;")
cursor.execute("DROP TABLE IF EXISTS par_mascotas;")
cursor.execute("DROP TABLE IF EXISTS par_especialista;")
cursor.execute("DROP TABLE IF EXISTS par_vacunas;")
cursor.execute("DROP TABLE IF EXISTS tbl_citas;")
cursor.execute("DROP TABLE IF EXISTS tbl_aplicacion_vacunas;")
conn.commit()

In [84]:
def Funcion_ejecutar_DML(query_dml, conexion=conn):
    """
    Funcion encargada de ejecutar  sentencias  de tipo DML(Manipulación de Datos - Manipula los datos dentro de las tablas - INSERT, UPDATE, DELETE) 
    """
    try:
        cursor = conexion.cursor()
        cursor.execute(query_dml)
        conexion.commit()
    except sqlite3.OperationalError as e:
        print(f" Se genera error  de ejecucion posiblemente  por que la columna ya existe {e}.")

    return f"DML ejecutado correctamente: {query_dml}"

Crear variables con estructura de las tablas

In [85]:
tabla_propietarios = """
                    CREATE TABLE IF NOT EXISTS par_propietario (
                            pk_propietario INTEGER PRIMARY KEY,
                            cedula TEXT NOT NULL UNIQUE,
                            nombre TEXT NOT NULL,
                            apellido TEXT NOT NULL,
                            direccion TEXT NOT NULL,
                            telefono TEXT NOT NULL,
                            fecha_creacion TEXT NOT NULL DEFAULT (datetime('now'))                    
                        );
                    """

tabla_mascotas = """
                    CREATE TABLE IF NOT EXISTS par_mascotas (
                            pk_mascota INTEGER PRIMARY KEY,
                            nombre TEXT NOT NULL,
                            especie TEXT NOT NULL,
                            raza TEXT NOT NULL,
                            fecha_nacimiento TEXT NOT NULL,                             
                            fk_propietario INTEGER NOT NULL,
                            fecha_creacion TEXT NOT NULL DEFAULT (datetime('now')),
                            FOREIGN KEY (fk_propietario) REFERENCES par_propietario(pk_propietario)
                        );
                    """

tabla_epecialistas = """
                    CREATE TABLE IF NOT EXISTS par_especialista (
                            pk_especialista INTEGER PRIMARY KEY,
                            cedula TEXT NOT NULL UNIQUE,
                            nombre TEXT NOT NULL,
                            apellido TEXT NOT NULL,
                            espcialidad TEXT NOT NULL,
                            telefono TEXT NOT NULL,
                            fecha_creacion TEXT NOT NULL DEFAULT (datetime('now'))                    
                        );
                    """
tabla_vacunas = """
                    CREATE TABLE IF NOT EXISTS par_vacunas (
                            pk_vacuna INTEGER PRIMARY KEY,
                            nombre TEXT NOT NULL,
                            mgl INTEGER NOT NULL,
                            concentracion TEXT NOT NULL,
                            contraindicaciones TEXT NOT NULL,
                            fecha_creacion TEXT NOT NULL DEFAULT (datetime('now'))
                        );
                    """

tabla_generacion_citas = """
                    CREATE TABLE IF NOT EXISTS tbl_citas (
                            pk_cita INTEGER PRIMARY KEY,
                            fk_mascota  INTEGER NOT NULL,
                            fk_especialista INTEGER NOT NULL,
                            motivo TEXT NOT NULL,
                            medicamentos_recetados TEXT,
                            fk_propietario INTEGER NOT NULL,
                            fecha_programada TEXT NOT NULL,
                            fecha_creacion TEXT NOT NULL DEFAULT (datetime('now')),
                            FOREIGN KEY (fk_mascota) REFERENCES par_mascotas(pk_mascota),
                            FOREIGN KEY (fk_especialista) REFERENCES par_especialista(pk_especialista),
                            FOREIGN KEY (fk_propietario) REFERENCES par_propietario(pk_propietario)
                        );
                    """

tabla_aplicacion_vacunas = """
                    CREATE TABLE IF NOT EXISTS tbl_aplicacion_vacunas (
                            pk_aplica_vacuna INTEGER PRIMARY KEY,
                            fk_mascota  INTEGER NOT NULL,
                            fk_especialista INTEGER NOT NULL,
                            fk_propietario INTEGER NOT NULL,
                            fk_vacuna INTEGER NOT NULL,
                            fecha_aplicacion TEXT NOT NULL,
                            fecha_creacion TEXT NOT NULL DEFAULT (datetime('now')),
                            FOREIGN KEY (fk_mascota) REFERENCES par_mascotas(pk_mascota),
                            FOREIGN KEY (fk_especialista) REFERENCES par_especialista(pk_especialista),
                            FOREIGN KEY (fk_propietario) REFERENCES par_propietario(pk_propietario),
                            FOREIGN KEY (fk_vacuna) REFERENCES par_vacunas(pk_vacuna)
                        );
                    """
 

Ejecutar funciones de tipo DDL
Ejecucion de las funciones de creacion de tablas FUNCION DDL

In [86]:
# Crear tabla de propietarios y mascotas
Funcion_ejecutar_DDL(tabla_propietarios)
Funcion_ejecutar_DDL(tabla_mascotas)
Funcion_ejecutar_DDL(tabla_epecialistas)
Funcion_ejecutar_DDL(tabla_generacion_citas)
Funcion_ejecutar_DDL(tabla_vacunas)
Funcion_ejecutar_DDL(tabla_aplicacion_vacunas)

"DDL ejecutado correctamente: \n                    CREATE TABLE IF NOT EXISTS tbl_aplicacion_vacunas (\n                            pk_aplica_vacuna INTEGER PRIMARY KEY,\n                            fk_mascota  INTEGER NOT NULL,\n                            fk_especialista INTEGER NOT NULL,\n                            fk_propietario INTEGER NOT NULL,\n                            fk_vacuna INTEGER NOT NULL,\n                            fecha_aplicacion TEXT NOT NULL,\n                            fecha_creacion TEXT NOT NULL DEFAULT (datetime('now')),\n                            FOREIGN KEY (fk_mascota) REFERENCES par_mascotas(pk_mascota),\n                            FOREIGN KEY (fk_especialista) REFERENCES par_especialista(pk_especialista),\n                            FOREIGN KEY (fk_propietario) REFERENCES par_propietario(pk_propietario),\n                            FOREIGN KEY (fk_vacuna) REFERENCES par_vacunas(pk_vacuna)\n                        );\n                    "

In [87]:
insetar_propietarios = """
                    INSERT INTO par_propietario(cedula,nombre,apellido,direccion,telefono) 
                    VALUES
                     ('1024878965','JUAN','PEREZ','CLL-25#63-78','3013195028')
                    ,('1032659453','MARIA','RODRIGUEZ','CLL-135#68-18','3204985722')
                    ,('789654265','ANDRES','GOMEZ','CLL-3#60-25','3154792325')
                    ,('982243256','HELENA','MARTINEZ','CLL-178#7A-26','3235927428')
"""
insertar_mascotas = """
                    INSERT INTO par_mascotas(nombre,	especie,	raza,	fecha_nacimiento,	fk_propietario)  
                    VALUES
                     ('LUNA','CANINO','PINCHER','2020-05-12',1)
                    ,('MANCHAS','CANINO','PITBULL','2015-12-10',2)
                    ,('MISIFU','GATO','ANGORA','2022-10-02',3)
                    ,('MOTAS','CANINO','FRENCHPUDLE','2019-08-20',4)
                """
insertar_especialistas = """
                            INSERT INTO par_especialista(cedula,nombre,apellido,espcialidad,telefono) 
                            VALUES
                             ('1082947847','DIEGO', 'SANCHEZ','VETERINARIO GENERAL','3156789856')
                            ,('1013456789','DIANA', 'HERNANDEZ','VETERINARIO VACUNAS','3156789856')
                            ,('1098765432','CARLOS','LOPEZ','VETERINARIO CIRUJANO','3204567890')
                            ,('1076543210','SOPHIA','RAMIREZ','VETERINARIO DERMATOLOGO','3204567890')
                        """

insertar_vacunas = """
                        INSERT INTO par_vacunas(nombre,mgl,concentracion,contraindicaciones) 
                        VALUES
                        ('Rabia (Rabies) — perros/gatos',5,'Dosis típica: 1 mL IM/SC (según producto)','Hipersensibilidad a componentes; animales con enfermedad febril grave. Seguir ficha técnica en gestación.')
                        ,('Moquillo canino (Distemper)',10,'Dosis típica: 1 mL SC (vacuna viva atenuada; esquema según edad)','No administrar a animales inmunodeprimidos o gravemente enfermos; precaución en hembras gestantes según producto.')
                        ,('Parvovirus canino',3,'Dosis típica: 1 mL SC (vacuna viva atenuada o recombinante)','Reacciones alérgicas previas; evitar en animales con enfermedad aguda.')
                        ,('Adenovirus canino (CAV-2 / hepatitis infecciosa)',8,'Normalmente incluida en vacunas combinadas (DHP/DHPP) — 1 mL SC','Hipersensibilidad conocida; animales gravemente enfermos aplazar.')
                        ,('Leptospirosis (canina)',10,'Dosis típica: 1 mL SC; suelen ser inactivadas (toxinas/antígenos)','Reacciones febriles o alérgicas previas; no administrar a animales enfermos; requiere esquema de refuerzo.')
                        ,('Bordetella bronchiseptica (tos de las perreras)',15,'Dosis típica: 0.51 mL (intranasal o SC/IM según formulación)','No aplicar en animales inmunodeprimidos; precaución en cachorros muy debilitados.')
                        ,('Panleucopenia felina (FPV)',15,'Dosis típica: 0.5 mL SC (vacuna viva atenuada en cachorros)','Evitar en gatos gravemente enfermos; reacciones alérgicas previas.')
                        ,('Calicivirus felino (FCV)',15,'Normalmente combinada (RCF: rabia opcional) — 0.5 mL SC','Hipersensibilidad a componentes; aplazar si el animal está enfermo.')
                        ,('Herpesvirus felino (Rinotraqueítis felina, FHV-1)',25,'Suele administrarse junto con FPV y FCV — 0.5 mL SC','Precaución en animales con enfermedad sistémica; alergia a componentes.')
                        ,('Leucemia felina (FeLV)',13,'Dosis típica: 0.5 mL SC — vacuna inactivada o recombinante; esquema en gatitos y adultos de riesgo','No administrar a gatos ya infectados (test previo recomendado); reacciones alérgicas previas; aplazar si el gato est')
                    
                    """
insertar_citas = """
                    INSERT INTO tbl_citas(
                    fk_mascota, fk_especialista, motivo, medicamentos_recetados,
                    fk_propietario, fecha_programada)                      
                    VALUES
                    (1,1,'Control general anual','Vitaminas básicas',1,'2025-01-10'),
                    (2,2,'Aplicación de vacuna parvovirus','Ninguno',2,'2025-01-15'),
                    (3,3,'Revisión respiratoria','Antiinflamatorio',3,'2025-01-20'),
                    (4,4,'Tratamiento dermatitis','Shampoo medicado',4,'2025-01-25'),
                    (1,2,'Vacunación múltiple DHPP','Ninguno',1,'2025-02-01'),
                    (2,1,'Pérdida de apetito','Protector gástrico',2,'2025-02-05'),
                    (3,4,'Irritación ocular','Gotas oftálmicas',3,'2025-02-08'),
                    (4,3,'Dolor en extremidad trasera','Analgésico suave',4,'2025-02-10'),
                    (1,3,'Evaluación prequirúrgica','Ninguno',1,'2025-02-15'),
                    (2,4,'Alergia cutánea','Antihistamínico',2,'2025-02-18'),
                    (3,1,'Chequeo postvacunación','Ninguno',3,'2025-03-01'),
                    (4,2,'Control de pulgas y garrapatas','Antiparasitario',4,'2025-03-03'),
                    (1,4,'Herida leve en pata','Crema cicatrizante',1,'2025-03-05'),
                    (2,3,'Tos persistente','Jarabe expectorante',2,'2025-03-09'),
                    (3,2,'Vacuna refuerzo FPV','Ninguno',3,'2025-03-12'),
                    (4,1,'Control general adulto','Vitaminas',4,'2025-03-15'),
                    (1,2,'Revisión de piel','Antihistamínico',1,'2025-03-18'),
                    (2,3,'Consulta nutricional','Ninguno',2,'2025-03-22'),
                    (3,4,'Problemas digestivos','Dieta blanda recomendada',3,'2025-03-25'),
                    (4,2,'Vacuna anual','Ninguno',4,'2025-03-28');
                """
insertar_aplicacion_vacunas = """
                                INSERT INTO tbl_aplicacion_vacunas(
                                    fk_mascota, fk_especialista, fk_propietario, fk_vacuna, fecha_aplicacion)
                                    VALUES
                                    (1,2,1,1,'2025-01-05'),
                                    (2,2,2,3,'2025-01-06'),
                                    (3,2,3,7,'2025-01-07'),
                                    (4,2,4,5,'2025-01-08'),
                                    (1,3,1,2,'2025-01-10'),
                                    (2,1,2,6,'2025-01-12'),
                                    (3,4,3,9,'2025-01-14'),
                                    (4,1,4,4,'2025-01-15'),
                                    (1,2,1,10,'2025-01-17'),
                                    (2,3,2,8,'2025-01-18'),
                                    (3,1,3,1,'2025-01-20'),
                                    (4,4,4,2,'2025-01-21'),
                                    (1,2,1,3,'2025-01-22'),
                                    (2,1,2,4,'2025-01-23'),
                                    (3,3,3,5,'2025-01-24'),
                                    (4,2,4,6,'2025-01-25'),
                                    (1,4,1,7,'2025-01-26'),
                                    (2,2,2,8,'2025-01-27'),
                                    (3,1,3,9,'2025-01-28'),
                                    (4,3,4,10,'2025-01-29'),
                                    (1,2,1,4,'2025-02-02'),
                                    (2,3,2,1,'2025-02-03'),
                                    (3,4,3,2,'2025-02-04'),
                                    (4,1,4,3,'2025-02-05'),
                                    (1,3,1,5,'2025-02-06'),
                                    (2,2,2,7,'2025-02-07'),
                                    (3,1,3,8,'2025-02-08'),
                                    (4,4,4,9,'2025-02-09'),
                                    (1,2,1,6,'2025-02-10'),
                                    (2,3,2,10,'2025-02-11');   
                                """



Ejecutar funcion de DML para la insercion de informacion de la clinica verterinaria

In [88]:
# Funcion_ejecutar_DML(insetar_propietarios)
# Funcion_ejecutar_DML(insertar_mascotas)    
# Funcion_ejecutar_DML(insertar_especialistas)
# Funcion_ejecutar_DML(insertar_vacunas)
Funcion_ejecutar_DML(insertar_citas)
Funcion_ejecutar_DML(insertar_aplicacion_vacunas)

"DML ejecutado correctamente: \n                                INSERT INTO tbl_aplicacion_vacunas(\n                                    fk_mascota, fk_especialista, fk_propietario, fk_vacuna, fecha_aplicacion)\n                                    VALUES\n                                    (1,2,1,1,'2025-01-05'),\n                                    (2,2,2,3,'2025-01-06'),\n                                    (3,2,3,7,'2025-01-07'),\n                                    (4,2,4,5,'2025-01-08'),\n                                    (1,3,1,2,'2025-01-10'),\n                                    (2,1,2,6,'2025-01-12'),\n                                    (3,4,3,9,'2025-01-14'),\n                                    (4,1,4,4,'2025-01-15'),\n                                    (1,2,1,10,'2025-01-17'),\n                                    (2,3,2,8,'2025-01-18'),\n                                    (3,1,3,1,'2025-01-20'),\n                                    (4,4,4,2,'2025-01-21'),\n           

In [89]:
df2 = Funcion_ejecutar_DQL("SELECT * from par_mascotas;")
display(df2)

,pk_mascota,nombre,especie,raza,fecha_nacimiento,fk_propietario,fecha_creacion


Ejecutar funciones de tipo DQL

In [90]:
df1 = Funcion_ejecutar_DQL("SELECT * from par_propietario;")
display(df1)
df2 = Funcion_ejecutar_DQL("SELECT * from par_mascotas;")
display(df2)
df3 = Funcion_ejecutar_DQL("SELECT * from par_especialista;")
display(df3)
df4 = Funcion_ejecutar_DQL("SELECT * from par_vacunas;")
display(df4)
##############################################################################
df5 = Funcion_ejecutar_DQL("SELECT * from tbl_citas;")
display(df5)
df6 = Funcion_ejecutar_DQL("SELECT * from tbl_aplicacion_vacunas;")
display(df6)    


,pk_propietario,cedula,nombre,apellido,direccion,telefono,fecha_creacion


,pk_mascota,nombre,especie,raza,fecha_nacimiento,fk_propietario,fecha_creacion


,pk_especialista,cedula,nombre,apellido,espcialidad,telefono,fecha_creacion


,pk_vacuna,nombre,mgl,concentracion,contraindicaciones,fecha_creacion


,pk_cita,fk_mascota,fk_especialista,motivo,medicamentos_recetados,fk_propietario,fecha_programada,fecha_creacion
0,1,1,1,Control general anual,Vitaminas básicas,1,2025-01-10,2025-11-26 03:46:02
1,2,2,2,Aplicación de vacuna parvovirus,Ninguno,2,2025-01-15,2025-11-26 03:46:02
2,3,3,3,Revisión respiratoria,Antiinflamatorio,3,2025-01-20,2025-11-26 03:46:02
3,4,4,4,Tratamiento dermatitis,Shampoo medicado,4,2025-01-25,2025-11-26 03:46:02
4,5,1,2,Vacunación múltiple DHPP,Ninguno,1,2025-02-01,2025-11-26 03:46:02
5,6,2,1,Pérdida de apetito,Protector gástrico,2,2025-02-05,2025-11-26 03:46:02
6,7,3,4,Irritación ocular,Gotas oftálmicas,3,2025-02-08,2025-11-26 03:46:02
7,8,4,3,Dolor en extremidad trasera,Analgésico suave,4,2025-02-10,2025-11-26 03:46:02
8,9,1,3,Evaluación prequirúrgica,Ninguno,1,2025-02-15,2025-11-26 03:46:02
9,10,2,4,Alergia cutánea,Antihistamínico,2,2025-02-18,2025-11-26 03:46:02


,pk_aplica_vacuna,fk_mascota,fk_especialista,fk_propietario,fk_vacuna,fecha_aplicacion,fecha_creacion
0,1,1,2,1,1,2025-01-05,2025-11-26 03:46:02
1,2,2,2,2,3,2025-01-06,2025-11-26 03:46:02
2,3,3,2,3,7,2025-01-07,2025-11-26 03:46:02
3,4,4,2,4,5,2025-01-08,2025-11-26 03:46:02
4,5,1,3,1,2,2025-01-10,2025-11-26 03:46:02
5,6,2,1,2,6,2025-01-12,2025-11-26 03:46:02
6,7,3,4,3,9,2025-01-14,2025-11-26 03:46:02
7,8,4,1,4,4,2025-01-15,2025-11-26 03:46:02
8,9,1,2,1,10,2025-01-17,2025-11-26 03:46:02
9,10,2,3,2,8,2025-01-18,2025-11-26 03:46:02
